### t-SNEによるペルソナプロンプトの２次元可視化

- 初期プロンプトから離れたか
- ユーザー同士の位置

In [1]:
from dotenv import load_dotenv
import os

from openai import AzureOpenAI
load_dotenv()

# 環境変数から設定を取得
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT')

embedding = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        api_version=os.getenv("AZURE_OPENAI_EMBEDDING_API_VERSION"),
        azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT"),
    )

# embeddings = embedding.embeddings.create(
#     model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT,
#     input=['hello']
# )
# embeddings.data[0].embedding

In [ ]:
# system_prompt を含む md ファイルを読み込み、Azure OpenAI Embeddings -> t-SNE 可視化
from pathlib import Path
import re
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px

base_dir = Path('/home/naofumi1014/PersonaAgent/experiment')
md_files = sorted(base_dir.glob('*systemprompt*.md'))
texts = []
labels = []
for f in md_files:
    try:
        content = f.read_text(encoding='utf-8')
    except Exception as e:
        print(f"読み込み失敗: {f}: {e}")
        continue
    norm = re.sub(r'\s+', ' ', content).strip()
    texts.append(norm)
    labels.append(f.stem)

print('対象ファイル:', labels)
embeddings = []
for t in texts:
    resp = embedding.embeddings.create(
        model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT,
        input=t
    )
    embeddings.append(resp.data[0].embedding)

embeddings = np.array(embeddings)
print('埋め込みshape:', embeddings.shape)

if len(embeddings) < 2:
    raise ValueError('可視化には2件以上のsystem promptファイルが必要です')

perplexity = max(2, min(30, len(embeddings)//2))
tsne = TSNE(n_components=2, random_state=42, init='pca', perplexity=perplexity)
points_2d = tsne.fit_transform(embeddings)

df = pd.DataFrame({'x': points_2d[:,0], 'y': points_2d[:,1], 'label': labels, 'text': texts})
fig = px.scatter(df, x='x', y='y', text='label', hover_data={'text': True}, title='System Prompt t-SNE')
fig.update_traces(textposition='top center')
fig.update_layout(
    width=700,
    height=700,
    yaxis=dict(scaleanchor="x", scaleratio=1)
)

fig.show()

対象ファイル: ['initial_systemprompt', 'userA_persona_systemprompt', 'userB_persona_systemprompt', 'userC_persona_systemprompt']
埋め込みshape: (4, 3072)
埋め込みshape: (4, 3072)


In [3]:
perplexity = max(2, min(30, len(embeddings)//2))
tsne = TSNE(n_components=2, random_state=42, init='pca', perplexity=perplexity)
points_2d = tsne.fit_transform(embeddings)

df = pd.DataFrame({'x': points_2d[:,0], 'y': points_2d[:,1], 'label': labels, 'text': texts})
fig = px.scatter(df, x='x', y='y', text='label', hover_data={'text': True}, title='System Prompt t-SNE')
fig.update_traces(textposition='top center')
fig.update_layout(
    width=700,
    height=700,
    yaxis=dict(scaleanchor="x", scaleratio=1)
)

fig.show()